# Анна Запорощенко

В рамках этого задания мы будем создавать программу, которая получая на вход отзыв, будет предсказывать, является ли отзыв положительным или отрицательным. Делать мы будем это таким образом: мы возьмём некоторое число заранее размеченных как положительные или отрицательные отзывов, выделим те слова, которые встречаются только в положительных или только в отрицательных отзывах, и будем считать, каких слов  в поступившем нам на проверку отзыве больше.


Мы будем работать по заранее определённому пайплайну:

1.  Сначала нам надо скачать дату -- соберите как минимум 60 (30 положительных  и 30 отрицательных) отзывов на похожие продукты (не надо мешать отзывы на отели с отзывами на ноутбуки) для составления " тонального словаря" (чем больше отзывов, тем лучше)  и 10 отзывов для проверки качества.   3 балла в случае сбора путём парсинга, 1 - если найдете уже готовые данные или просто закопипастите без парсинга

2. Токенизируйте слова,  приведите их к нижнему регистру и к начальной форме  (1 балл за токенизацию, 1 - за начальную форму)

3. Составьте 2 множества - в одном будут слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных. Попробуйте поиграть с частотностями и исключить шум (к примеру, выбросить слова, встречающиеся 1-2 раза) (3 балла) (если у вас получились пустые множества, уберите фильтр по частотности или увеличьте выборку)

4. Создайте функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи accuracy (1  - за коректно работающую функцию, 1 - за подсчёт accuracy)

5. Предложите как минимум 2 способа улучшить эту программу с помощью добавления к ней любых мулек  - просто словами, писать улучшающий код не надо (1 балл)

6. Логичность и чистота кода (1 балл) 

Да, тут 12 баллов - два можно потерять.


В случае, если после долгих мучений в п. 3 множества по объективным причинам не получается (покажите, что пытались) - отправляйте жабу - зачьтём полный балл

In [64]:
import urllib.request
from fake_useragent import UserAgent
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from pymystem3 import Mystem
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from collections import Counter
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\VADIK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\VADIK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [65]:
segmenter = Segmenter()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

In [66]:
#Функция очищает список текстов от тегов и ненужных знаков
def parse(info: list) -> list:
    reviews = []
    for i in info:
        n = re.sub('<[A-Za-z\/][^>]*>|\r', '', i[1])
        reviews.append(re.sub('\n', '. ', n))
    return reviews

In [67]:
#Функция принимает ссылку и ищет сначала все отзывы с самой низкой оценкой, затем - с самой высокой. 
#Возвращает по 44 отзыва каждого вида общим списком.
def get_reviews(url: str) -> list:
    user_agent = UserAgent().chrome
    req = urllib.request.Request(url, headers={'User-Agent':user_agent})
    with urllib.request.urlopen(req) as response:
        html_content = response.read().decode('utf-8')
    bad_reviews = parse(re.findall('<b>1</b></span> из 10(.+?)jrReviewComment"><div >(.+?)</div>', html_content, re.DOTALL))
    good_reviews = parse(re.findall('<b>10</b></span> из 10(.+?)jrReviewComment"><div >(.+?)</div>', html_content, re.DOTALL))
    return bad_reviews[0:44] + good_reviews[0:44]

In [68]:
#Функция принимает список отзывов. Каждый отзыв заменяет на список лемм, которые точно состоят 
#из букв и не являются стоп-словами. Все слова приводятся к нижнему регистру. Функция возвращает матрицу
#со списками лемм для каждого отзыва.
#Вряд ли сработает на Google Colab, потому что Mystem.lemmatize() подвисает на нем
def preprocessing(reviews: list) -> list:
    russian_stopwords = stopwords.words('russian')
    m = Mystem()
    reviews_new = []
    for review in tqdm(reviews):
        review_new = []
        for word in m.lemmatize(review):
            if (word.isalpha())and(word not in russian_stopwords):
                review_new.append(word.lower())
        reviews_new.append(review_new)
    return reviews_new

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!NEW CODE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [76]:
def preprocessing_2(reviews: list, token_reviews: list) -> list:
    m = Mystem()
    for r in tqdm(enumerate(reviews)):
        doc = Doc(r[1])
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        doc.parse_syntax(syntax_parser)
        heads = []
        for token in doc.tokens:
            if (token.pos == 'NOUN')or(token.pos == 'ADJ')or(token.pos == 'PREP'):
                heads.append(token.id)
        for head in heads:
            collacation = ''
            for token in doc.tokens:
                if ((token.id == head)or(token.head_id == head))and(token.text.isalpha() == True):
                    collacation += m.lemmatize(token.text.lower())[0] + ' '
            if len(collacation.strip(' ').split(' ')) > 1:
                token_reviews[r[0]].append(collacation.strip(' '))
    return token_reviews

In [70]:
reviews = get_reviews('https://www.megacritic.ru/film/prometej')

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "C:\Users\VADIK\anaconda3\lib\urllib\request.py", line 1350, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "C:\Users\VADIK\anaconda3\lib\http\client.py", line 1240, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "C:\Users\VADIK\anaconda3\lib\http\client.py", line 1286, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "C:\Users\VADIK\anaconda3\lib\http\client.py", line 1235, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "C:\Users\VADIK\anaconda3\lib\http\client.py", line 1006, in _send_output
    self.send(msg)
  File "C:\Users\VADIK\anaconda3\lib\http\client.py", line 946, in send
    self.connect()
  File "C:\Users\VADIK\anaconda3\lib\http\client.py", line 917, in connect
    self.so

In [71]:
token_reviews = preprocessing(reviews)

100%|██████████████████████████████████████████████████████████████████████████████████| 88/88 [01:42<00:00,  1.17s/it]


In [77]:
collacations_and_words = preprocessing_2(reviews, token_reviews)

88it [2:05:01, 85.24s/it] 


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!NEW CODE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [105]:
X_train, X_test, y_train, y_test = train_test_split(np.array(collacations_and_words),
                                                    np.concatenate((np.zeros(44), np.ones(44))),
                                                    test_size=10/88,
                                                    random_state=0)

In [106]:
#Списки лемм из отрицательных и положительных отзывов X_train помещаются в два отдельных списка, 
#из них потом создаются частотные словари.Также создается частотный словарь по всем отзывам X_train.
bad_reviews = []
good_reviews = []
for y in enumerate(y_train):
    if y[1] == 0:
        bad_reviews.extend(X_train[y[0]])
    else:
        good_reviews.extend(X_train[y[0]])
bad_reviews = Counter(bad_reviews)
good_reviews = Counter(good_reviews)
reviews = Counter(sum(X_train.tolist(), []))

In [109]:
#Функция принимает частотные словари отрицательных и положительных отзывов, а также общий частотный словарь.
#Она создает словарь со словами, которые присутствуют и в положительных, и в отрицательных отзывах.
def check (bad_reviews: Counter, good_reviews: Counter, reviews: Counter) -> Counter:
    words = Counter()
    for word, value in reviews.items():
        if (word in bad_reviews.keys())and(word in good_reviews.keys()):
            words[word] = value
    return words

In [110]:
#Функция позволяет регулировать, насколько часто встречающиеся слова отправлять в функцию check.
def cutter (reviews: Counter, start: int, stop: int) -> Counter:
    new_reviews = Counter()
    for word, value in reviews.items():
        if (value >= start)and(value <= stop):
            new_reviews[word] = value
    return new_reviews

Можно проверить, есть ли какая-то частотность, при которой конкретная лемма встречается только в одном виде отзывов. Единица не дает нам какой-либо информации, с частотностью 19 на самом деле есть только 1 слово.

In [111]:
numbers = list(Counter(reviews.values()).keys())
numbers.sort()
for number in numbers:
    print('частотность слова ', number, ' количество повторов ', 
          len(check(bad_reviews, good_reviews, cutter(reviews, number, number))))

частотность слова  1  количество повторов  0
частотность слова  2  количество повторов  148
частотность слова  3  количество повторов  82
частотность слова  4  количество повторов  59
частотность слова  5  количество повторов  30
частотность слова  6  количество повторов  26
частотность слова  7  количество повторов  16
частотность слова  8  количество повторов  11
частотность слова  9  количество повторов  6
частотность слова  10  количество повторов  5
частотность слова  11  количество повторов  11
частотность слова  12  количество повторов  4
частотность слова  13  количество повторов  3
частотность слова  14  количество повторов  3
частотность слова  15  количество повторов  2
частотность слова  17  количество повторов  5
частотность слова  18  количество повторов  1
частотность слова  19  количество повторов  0
частотность слова  20  количество повторов  2
частотность слова  22  количество повторов  2
частотность слова  23  количество повторов  2
частотность слова  24  количество 

Зато можно увидеть, что слова перестают создавать больше количество "шума", когда встречаются более 6 раз. Ровно разделить все слова на "положительные" и "отрицательные" не получилось.

In [118]:
neutral = check(bad_reviews, good_reviews, reviews)

In [129]:
def get_words(neutral: Counter, polar: Counter) -> Counter:
    new = Counter()
    for word, value in polar.items():
        if word not in neutral: #убираем все слова с частотностью меньше 3
            new[word] = value
    return new

In [130]:
bad_reviews = get_words(neutral, bad_reviews)
good_reviews = get_words(neutral, good_reviews)

In [131]:
#Функция проверяет каждое слово в отдельном отзыве.
#Если слово есть в словаре негативных отзывов, score получает -1, если в словаре положительных - +1.
#Таким образом, каждый отзыв насчитывает свой score, в зависимости от которого происходит предсказание.
def predict(X_test: np.array, bad_reviews: Counter, good_reviews: Counter) -> np.array:
    results = []
    for review in X_test:
        score = 0
        for word in review:
            if word in bad_reviews.keys():
                score -= 1
            elif word in good_reviews.keys():
                score +=1
            else:
                pass
        if score < 0:
            results.append(0)
        else:
            results.append(1)
    return np.array(results)

In [132]:
accuracy_score(predict(X_test, bad_reviews, good_reviews), y_test)

0.6

Ничего не изменилось 😭

# Предложения
1) Учитывать n-граммы.
2) "Нейтральные" слова тоже могли бы быть полезными: если слово встретилось 1 раз в положительных отзывах и 40 в отрицательных, исключать его при этом объеме данных невыгодно.